In [1]:
import pandas as pd
import numpy as np
from datetime import date

In [2]:
import duckdb

In [3]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

In [4]:
from joblib import dump

# 1. Load data

In [5]:
df_info = pd.read_csv('cc_info.csv')
df_txn = pd.read_csv('transactions.csv')

# 2. Check data quality

In [6]:
df_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 984 entries, 0 to 983
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   credit_card        984 non-null    int64 
 1   city               984 non-null    object
 2   state              984 non-null    object
 3   zipcode            984 non-null    int64 
 4   credit_card_limit  984 non-null    int64 
dtypes: int64(3), object(2)
memory usage: 38.6+ KB


In [7]:
df_txn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294588 entries, 0 to 294587
Data columns (total 5 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   credit_card                294588 non-null  int64  
 1   date                       294588 non-null  object 
 2   transaction_dollar_amount  294588 non-null  float64
 3   Long                       294588 non-null  float64
 4   Lat                        294588 non-null  float64
dtypes: float64(3), int64(1), object(1)
memory usage: 11.2+ MB


In [8]:
df_txn['date'] = pd.to_datetime(df_txn['date'])

# 3. Exploratory data analysis (EDA)

df_info

In [9]:
duckdb.query("""
select *
from df_info
limit 5
""")

┌──────────────────┬──────────┬─────────┬─────────┬───────────────────┐
│   credit_card    │   city   │  state  │ zipcode │ credit_card_limit │
│      int64       │ varchar  │ varchar │  int64  │       int64       │
├──────────────────┼──────────┼─────────┼─────────┼───────────────────┤
│ 1280981422329509 │ Dallas   │ PA      │   18612 │              6000 │
│ 9737219864179988 │ Houston  │ PA      │   15342 │             16000 │
│ 4749889059323202 │ Auburn   │ MA      │    1501 │             14000 │
│ 9591503562024072 │ Orlando  │ WV      │   26412 │             18000 │
│ 2095640259001271 │ New York │ NY      │   10001 │             20000 │
└──────────────────┴──────────┴─────────┴─────────┴───────────────────┘

In [10]:
duckdb.query("""
select count(*), count(distinct credit_card)
from df_info
""")

┌──────────────┬─────────────────────────────┐
│ count_star() │ count(DISTINCT credit_card) │
│    int64     │            int64            │
├──────────────┼─────────────────────────────┤
│          984 │                         984 │
└──────────────┴─────────────────────────────┘

In [11]:
duckdb.query("""
select count(distinct credit_card) no_cust
, count(distinct city) no_city
, count(distinct state) no_state
, count(distinct zipcode) no_zipcode
, avg(credit_card_limit) avg_cc_ilm
from df_info
""")

┌─────────┬─────────┬──────────┬────────────┬────────────────────┐
│ no_cust │ no_city │ no_state │ no_zipcode │     avg_cc_ilm     │
│  int64  │  int64  │  int64   │   int64    │       double       │
├─────────┼─────────┼──────────┼────────────┼────────────────────┤
│     984 │     124 │       35 │        124 │ 12321.138211382115 │
└─────────┴─────────┴──────────┴────────────┴────────────────────┘

In [12]:
duckdb.query("""
select city
, count(*) no_cust
, avg(credit_card_limit) avg_cc_lim
from df_info
group by city
order by avg(credit_card_limit) desc
limit 10
""")

┌────────────┬─────────┬────────────┐
│    city    │ no_cust │ avg_cc_lim │
│  varchar   │  int64  │   double   │
├────────────┼─────────┼────────────┤
│ Monticello │       1 │    42000.0 │
│ Fort Wayne │       1 │    30000.0 │
│ Tampa      │       2 │    30000.0 │
│ Cleveland  │       2 │    25000.0 │
│ Knoxville  │       1 │    25000.0 │
│ Amarillo   │       1 │    24000.0 │
│ Raleigh    │       2 │    22500.0 │
│ Oakland    │       2 │    22500.0 │
│ Garfield   │       1 │    21000.0 │
│ Denver     │       4 │    20000.0 │
├────────────┴─────────┴────────────┤
│ 10 rows                 3 columns │
└───────────────────────────────────┘

In [13]:
duckdb.query("""
select zipcode
, count(*) no_cust
, avg(credit_card_limit) avg_cc_lim
from df_info
group by zipcode
order by avg(credit_card_limit) desc
limit 10
""")

┌─────────┬─────────┬────────────┐
│ zipcode │ no_cust │ avg_cc_lim │
│  int64  │  int64  │   double   │
├─────────┼─────────┼────────────┤
│    4760 │       1 │    42000.0 │
│   33601 │       2 │    30000.0 │
│   46801 │       1 │    30000.0 │
│   13042 │       2 │    25000.0 │
│   16928 │       1 │    25000.0 │
│   79101 │       1 │    24000.0 │
│   25911 │       2 │    22500.0 │
│    2858 │       2 │    22500.0 │
│    7026 │       1 │    21000.0 │
│   12421 │       4 │    20000.0 │
├─────────┴─────────┴────────────┤
│ 10 rows              3 columns │
└────────────────────────────────┘

In [14]:
duckdb.query("""
select state
, count(*) no_cust
, avg(credit_card_limit) avg_cc_lim
from df_info
group by state
order by avg(credit_card_limit) desc
limit 10
""")

┌─────────┬─────────┬────────────────────┐
│  state  │ no_cust │     avg_cc_lim     │
│ varchar │  int64  │       double       │
├─────────┼─────────┼────────────────────┤
│ FL      │       4 │            20500.0 │
│ AZ      │       1 │            20000.0 │
│ IN      │       9 │            17000.0 │
│ IA      │       4 │            16000.0 │
│ RI      │       5 │            15400.0 │
│ PR      │       8 │            15125.0 │
│ WV      │      23 │ 14695.652173913044 │
│ GA      │      12 │ 14666.666666666666 │
│ AL      │       3 │ 14333.333333333334 │
│ OH      │      11 │            14000.0 │
├─────────┴─────────┴────────────────────┤
│ 10 rows                      3 columns │
└────────────────────────────────────────┘

df_txn

In [15]:
duckdb.query("""
select *
from df_txn
limit 5
""")

┌──────────────────┬─────────────────────┬───────────────────────────┬───────────────────┬──────────────────┐
│   credit_card    │        date         │ transaction_dollar_amount │       Long        │       Lat        │
│      int64       │    timestamp_ns     │          double           │      double       │      double      │
├──────────────────┼─────────────────────┼───────────────────────────┼───────────────────┼──────────────────┤
│ 1003715054175576 │ 2015-09-11 00:32:40 │                     43.78 │ -80.1741323292432 │ 40.2673703430082 │
│ 1003715054175576 │ 2015-10-24 22:23:08 │                    103.15 │ -80.1942395924575 │ 40.1801138227341 │
│ 1003715054175576 │ 2015-10-26 18:19:36 │                     48.55 │ -80.2110325106832 │  40.313004401402 │
│ 1003715054175576 │ 2015-10-22 19:41:10 │                    136.18 │ -80.1741379707697 │ 40.2908953862502 │
│ 1003715054175576 │ 2015-10-26 20:08:22 │                     71.82 │ -80.2387197193347 │ 40.1667191961412 │
└─────────

In [16]:
duckdb.query("""
select count(distinct credit_card) no_cust
, avg(transaction_dollar_amount) avg_txn_amt
, avg(Long) avg_long
, avg(Lat) avg_lat
from df_txn
limit 5
""")

┌─────────┬───────────────────┬────────────────────┬───────────────────┐
│ no_cust │    avg_txn_amt    │      avg_long      │      avg_lat      │
│  int64  │      double       │       double       │      double       │
├─────────┼───────────────────┼────────────────────┼───────────────────┤
│     984 │ 86.00803617255285 │ -76.23523815998024 │ 40.93766857401737 │
└─────────┴───────────────────┴────────────────────┴───────────────────┘

In [17]:
duckdb.query("""
select distinct date
from df_txn
order by date
""")

┌────────────────────────┐
│          date          │
│      timestamp_ns      │
├────────────────────────┤
│ 2015-07-31 09:39:48    │
│ 2015-07-31 11:03:48    │
│ 2015-07-31 11:10:14    │
│ 2015-07-31 11:28:55    │
│ 2015-07-31 11:38:51    │
│ 2015-07-31 11:53:15    │
│ 2015-07-31 11:59:10    │
│ 2015-07-31 12:00:50    │
│ 2015-07-31 12:01:41    │
│ 2015-07-31 12:02:59    │
│          ·             │
│          ·             │
│          ·             │
│ 2015-08-02 22:53:58    │
│ 2015-08-02 22:54:17    │
│ 2015-08-02 22:54:32    │
│ 2015-08-02 22:54:56    │
│ 2015-08-02 22:54:59    │
│ 2015-08-02 22:55:01    │
│ 2015-08-02 22:55:07    │
│ 2015-08-02 22:55:09    │
│ 2015-08-02 22:55:18    │
│ 2015-08-02 22:55:24    │
├────────────────────────┤
│         ? rows         │
│ (>9999 rows, 20 shown) │
└────────────────────────┘

# 4. Feature engineering

create function

In [18]:
# ytd is day -1
# td is day 0
# tmr is day 1

In [19]:
def gen_label(feat_dt, txn):
  txn['tmp_date'] = feat_dt
  txn['tmp_date'] = pd.to_datetime(txn['tmp_date'])
  txn['day_diff'] = (txn['date']-txn['tmp_date']).dt.days
  label = txn[txn['day_diff'].between(1, 7)].groupby('credit_card').agg(sum_amt_label=('transaction_dollar_amount', 'sum')).reset_index()
  label['sum_amt_label'] = label['sum_amt_label'].fillna(0)
  label['feat_dt'] = feat_dt
  return label

In [20]:
# tmr is day -1
# td is day 0
# ytd is day 1

In [21]:
def gen_feat_sum_amount(feat_dt, txn):
    txn['tmp_date'] = feat_dt
    txn['tmp_date'] = pd.to_datetime(txn['tmp_date'])
    txn['day_diff'] = (txn['tmp_date'] - txn['date']).dt.days

    feat_1 = txn[txn['day_diff'].between(0, 7)].groupby('credit_card').agg(
        sum_amt_l1w=('transaction_dollar_amount', 'sum'),
        mean_amt_l1w=('transaction_dollar_amount', 'mean'),
        min_amt_l1w=('transaction_dollar_amount', 'min'),
        max_amt_l1w=('transaction_dollar_amount', 'max')
    ).reset_index()
    
    feat_2 = txn[txn['day_diff'].between(0, 14)].groupby('credit_card').agg(
        sum_amt_l2w=('transaction_dollar_amount', 'sum'),
        mean_amt_l2w=('transaction_dollar_amount', 'mean'),
        min_amt_l2w=('transaction_dollar_amount', 'min'),
        max_amt_l2w=('transaction_dollar_amount', 'max')
    ).reset_index()
    
    feat_3 = txn[txn['day_diff'].between(0, 28)].groupby('credit_card').agg(
        sum_amt_l4w=('transaction_dollar_amount', 'sum'),
        mean_amt_l4w=('transaction_dollar_amount', 'mean'),
        min_amt_l4w=('transaction_dollar_amount', 'min'),
        max_amt_l4w=('transaction_dollar_amount', 'max')
    ).reset_index()

    feat_sum_amt = (
        feat_1.merge(feat_2, on='credit_card', how='outer')
              .merge(feat_3, on='credit_card', how='outer')
    )
    feat_sum_amt['feat_dt'] = feat_dt
    feat_sum_amt = feat_sum_amt.fillna(0)
    
    return feat_sum_amt

In [22]:
def gen_feat_count_txn(feat_dt, txn):
  txn['tmp_date'] = feat_dt
  txn['tmp_date'] = pd.to_datetime(txn['tmp_date'])
  txn['day_diff'] = (txn['tmp_date']-txn['date']).dt.days

  feat_1 = txn[txn['day_diff'].between(0, 7)].groupby('credit_card').agg(count_txn_l1w=('transaction_dollar_amount', 'count')).reset_index()
  feat_2 = txn[txn['day_diff'].between(0, 14)].groupby('credit_card').agg(count_txn_l2w=('transaction_dollar_amount', 'count')).reset_index()
  feat_3 = txn[txn['day_diff'].between(0, 28)].groupby('credit_card').agg(count_txn_l4w=('transaction_dollar_amount', 'count')).reset_index()

  feat_count_txn = feat_1.merge(feat_2, on='credit_card', how='outer').merge(feat_3, on='credit_card', how='outer')
  feat_count_txn['feat_dt'] = feat_dt
  feat_count_txn = feat_count_txn.fillna(0)
  return feat_count_txn

In [23]:
def gen_feat_usage(feat_dt, txn, card_info):
  txn['tmp_date'] = feat_dt
  txn['tmp_date'] = pd.to_datetime(txn['tmp_date'])
  txn['day_diff'] = (txn['tmp_date']-txn['date']).dt.days

  feat_1 = txn[txn['day_diff'].between(0, 28)].groupby('credit_card').agg(sum_amt_l1m=('transaction_dollar_amount', 'sum')).reset_index()
  feat_sum_amt = feat_1.fillna(0)
  feat_sum_amt['feat_dt'] = feat_dt

  feat_pct_usage = feat_sum_amt.merge(card_info[['credit_card', 'credit_card_limit']], on='credit_card')
  feat_pct_usage['pct_usage'] = feat_pct_usage['sum_amt_l1m'] * 100 / feat_pct_usage['credit_card_limit']
  feat_pct_usage = feat_pct_usage.drop(columns=['credit_card_limit'])

  return feat_pct_usage

In [24]:
def gen_feat_average_latlong(feat_dt, txn):
    txn['tmp_date'] = feat_dt
    txn['tmp_date'] = pd.to_datetime(txn['tmp_date'])
    txn['day_diff'] = (txn['tmp_date'] - txn['date']).dt.days

    filtered_txn = txn[txn['day_diff'].between(0, 999999)]

    feat_avg_latlong = filtered_txn.groupby('credit_card').agg(
        avg_long=('Long', 'mean'),
        avg_lat=('Lat', 'mean')
    ).reset_index()

    feat_avg_latlong = feat_avg_latlong.fillna(0)

    feat_avg_latlong['feat_dt'] = feat_dt

    return feat_avg_latlong

In [25]:
def gen_feat_location(info):
  info_sorted = info.sort_values(by='zipcode', ascending=True)
  info_sorted_unique = info_sorted.drop_duplicates(keep='first')
  info_sorted_unique = info_sorted_unique[['credit_card','city','state']]

  return info_sorted_unique

In [26]:
# combine

In [27]:
def gen_feat_label(feat_dt, info, txn):

  txn['date'] = pd.to_datetime(txn['date'])

  f1 = gen_feat_sum_amount(feat_dt, txn)
  f2 = gen_feat_count_txn(feat_dt, txn)
  f3 = gen_feat_usage(feat_dt, txn, info)
  f4 = gen_feat_average_latlong(feat_dt, txn)
  f5 = gen_feat_location(info)

  all_feat = f1.merge(
      f2, on=['credit_card', 'feat_dt'], how='outer'
  ).merge(
      f3, on=['credit_card', 'feat_dt'], how='outer'
  ).merge(
      f4, on=['credit_card', 'feat_dt'], how='outer'
  ) .merge(
      f5, on=['credit_card'], how='outer'
  )

  label = gen_label(feat_dt, txn)
  all_feat = all_feat.merge(label, on=['credit_card', 'feat_dt'], how='outer')

  return all_feat[[
      'credit_card', 'feat_dt',
      'sum_amt_l1w', 'mean_amt_l1w', 'min_amt_l1w', 'max_amt_l1w',
      'sum_amt_l2w', 'mean_amt_l2w', 'min_amt_l2w', 'max_amt_l2w',
      'sum_amt_l4w', 'mean_amt_l4w', 'min_amt_l4w', 'max_amt_l4w',
      'count_txn_l1w', 'count_txn_l2w', 'count_txn_l4w',
      'pct_usage',
      'avg_long', 'avg_lat','sum_amt_label',
      'city','state']]

gen function

In [28]:
x1 = gen_feat_label(feat_dt=date(2015, 8, 31), info=df_info, txn=df_txn)
x2 = gen_feat_label(feat_dt=date(2015, 9, 7), info=df_info, txn=df_txn)
x3 = gen_feat_label(feat_dt=date(2015, 9, 14), info=df_info, txn=df_txn)
x4 = gen_feat_label(feat_dt=date(2015, 9, 21), info=df_info, txn=df_txn)
x5 = gen_feat_label(feat_dt=date(2015, 9, 28), info=df_info, txn=df_txn)
x6 = gen_feat_label(feat_dt=date(2015, 10, 7), info=df_info, txn=df_txn)
x7 = gen_feat_label(feat_dt=date(2015, 10, 14), info=df_info, txn=df_txn)
x8 = gen_feat_label(feat_dt=date(2015, 10, 21), info=df_info, txn=df_txn)
x9 = gen_feat_label(feat_dt=date(2015, 10, 28), info=df_info, txn=df_txn)

In [29]:
df_final = pd.concat([x1, x2, x3, x4, x5, x6, x7, x8, x9])

clean data

In [30]:
df_final = duckdb.query("""
select *
from df_final
where credit_card is not null
and feat_dt is not null and city is not null and state is not null
""").to_df()

In [31]:
df_final.fillna(0, inplace = True)

In [32]:
df_final.head()

,credit_card,feat_dt,sum_amt_l1w,mean_amt_l1w,min_amt_l1w,max_amt_l1w,sum_amt_l2w,mean_amt_l2w,min_amt_l2w,max_amt_l2w,...,max_amt_l4w,count_txn_l1w,count_txn_l2w,count_txn_l4w,pct_usage,avg_long,avg_lat,sum_amt_label,city,state
0,1003715054175576,2015-08-31,2848.51,123.848261,13.43,859.31,4280.81,112.652895,13.43,859.31,...,859.31,23.0,38.0,80.0,39.397100,-80.216954,40.240437,3116.94,Houston,PA
1,1013870087888817,2015-08-31,3062.95,109.391071,0.70,205.01,6577.99,119.599818,0.70,972.63,...,972.63,28.0,55.0,107.0,74.525800,-68.910456,42.747861,4113.85,Washington,NH
2,1023820165155391,2015-08-31,5810.16,116.203200,0.46,939.07,12283.20,127.950000,0.46,977.59,...,977.59,50.0,96.0,186.0,78.105179,-72.175743,43.942523,4917.81,Charlotte,VT
3,1073931538936472,2015-08-31,566.18,113.236000,76.77,140.36,1014.47,126.808750,76.77,198.65,...,235.51,5.0,8.0,21.0,29.087200,-71.722386,41.607113,355.85,Dallas,PA
4,1077622576192810,2015-08-31,122.93,122.930000,122.93,122.93,278.07,139.035000,122.93,155.14,...,160.11,1.0,2.0,5.0,6.346100,-80.214683,40.260375,0.00,Houston,PA


# 5. Model

train test split

In [33]:
start_train = pd.to_datetime(date(2015, 8, 31))
end_train = pd.to_datetime(date(2015, 10, 14))

start_test = pd.to_datetime(date(2015, 10, 21))
end_test = pd.to_datetime(date(2015, 10, 28))

In [34]:
df_train = df_final[df_final['feat_dt'].between(start_train, end_train)]

df_test = df_final[df_final['feat_dt'].between(start_test, end_test)]

In [35]:
df_train.drop(columns=['credit_card','feat_dt'], inplace=True)

df_test.drop(columns=['credit_card','feat_dt'], inplace=True)

C:\Users\user\AppData\Local\Temp\ipykernel_11220\2125116413.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train.drop(columns=['credit_card','feat_dt'], inplace=True)
C:\Users\user\AppData\Local\Temp\ipykernel_11220\2125116413.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test.drop(columns=['credit_card','feat_dt'], inplace=True)


In [36]:
X_train = df_train.drop(columns=['sum_amt_label'])
y_train = df_train['sum_amt_label']

X_test = df_test.drop(columns=['sum_amt_label'])
y_test = df_test['sum_amt_label']

one hot encoder

In [37]:
encoder = OneHotEncoder(drop='first')

In [38]:
X_train_encoded = encoder.fit_transform(X_train[['city', 'state']])
X_test_encoded = encoder.transform(X_test[['city', 'state']])

In [39]:
X_train_encoded = pd.DataFrame(X_train_encoded.toarray(), columns=encoder.get_feature_names_out(['city', 'state']))
X_test_encoded = pd.DataFrame(X_test_encoded.toarray(), columns=encoder.get_feature_names_out(['city', 'state']))

In [40]:
X_train_encoded = pd.concat([X_train.drop(columns=['city', 'state']).reset_index(drop=True), X_train_encoded.reset_index(drop=True)], axis=1)
X_test_encoded = pd.concat([X_test.drop(columns=['city', 'state']).reset_index(drop=True), X_test_encoded.reset_index(drop=True)], axis=1)

train model

In [41]:
rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42)

In [42]:
rf_regressor.fit(X_train_encoded, y_train)

RandomForestRegressor(random_state=42)

In [43]:
y_pred_train = rf_regressor.predict(X_train_encoded)
y_pred_test = rf_regressor.predict(X_test_encoded)

In [44]:
train_mae = mean_absolute_error(y_train, y_pred_train)
test_mae = mean_absolute_error(y_test, y_pred_test)

In [45]:
print(f"Train MAE: {train_mae}")
print(f"Test MAE: {test_mae}")

Train MAE: 213.49380941852962
Test MAE: 1088.8276767153018
